In [1]:
import sys
import os
import pickle
from datetime import datetime 

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split 
from sklearn.feature_extraction import DictVectorizer 
from sklearn.metrics import roc_auc_score, log_loss, root_mean_squared_error
import xgboost as xgb
from hyperopt import fmin, hp, tpe, Trials
from hyperopt.pyll import scope

import mlflow

from helpers import cfg, supports, load_transform_predict

In [2]:
# initialize configuration
# mode=test will minimize parameters (ex: 20 VS 1000 booster runs)
# skip_optimization=True will skip model tuning
CONFIG = cfg.init_config(mode="test", skip_optimization=False)

# output config
print("---CONFIG:")
for k, v in CONFIG.items():
    print(f"---{k} > {v}")

---CONFIG:
---mode > test
---mlflow_evals_nbr > 2
---booster_rounds > 10
---skip_optimization > False
---mlflow_artifacts_path > /home/adi/projects/political-engagement-mlops/mlflow/mlruns/poleng


In [3]:
# # launch mlflow
# mlflow ui --backend-store-uri sqlite:///mlflow/mlflow.db --default-artifact-root mlflow
experiment = "political_engagement"
artifact_location = CONFIG["mlflow_artifacts_path"]

mlflow.set_tracking_uri("sqlite:///mlflow/mlflow.db")
mlflow.set_experiment(experiment)
supports.set_mlflow_artifact_location(
    "./mlflow/mlflow.db",
    experiment,
    artifact_location
)

In [4]:
# read data
data = load_transform_predict.Loader.load_live_data()

Loaded: ./data/training_data/live_data.parquet


In [5]:
# setup training context
# get datasets
dftrain, dftest = train_test_split(data, test_size=0.2, random_state=99)

# get targets
ytrain = dftrain["political_engagement"].values
ytest = dftest["political_engagement"].values
dftrain.drop(columns=["political_engagement"], inplace=True)
dftest.drop(columns=["political_engagement"], inplace=True)

# vectorize
dv = DictVectorizer(sparse=False)
train_dict = dftrain.to_dict(orient="records")
test_dict = dftest.to_dict(orient="records")
xtrain = dv.fit_transform(train_dict)
xtest = dv.transform(test_dict)

# get dmatrix
xtrain = xgb.DMatrix(xtrain, label=ytrain)
xtest = xgb.DMatrix(xtest, label=ytest)

In [6]:
# optimize
if not CONFIG["skip_optimization"]:
    search_space = {
        "learning_rate": hp.loguniform("learning_rate", -7, 10),
        "max_depth": scope.int(hp.quniform("max_depth", 0, 100, 1)),
        "min_child_weight": hp.loguniform("min_child_weight", -1, 4.6),
        "reg_alpha": hp.loguniform("reg_alpha", -5, 4.6), 
        "scale_pos_weight": hp.loguniform("scale_pos_weight", 0, 4.6),
        "objective": "binary:logistic",
        "seed": 99
    }

    best_result = fmin(
        fn=lambda search_space: supports.objective(
            search_space=search_space,
            xtrain=xtrain,
            xtest=xtest,
            ytrain=ytrain,
            ytest=ytest,
            num_boost_round=CONFIG["booster_rounds"]
            ),
        space=search_space,
        algo=tpe.suggest,
        max_evals=CONFIG["mlflow_evals_nbr"],
        trials=Trials()
        )

[0]	test-logloss:0.55135                             
[1]	test-logloss:0.53217                             
[2]	test-logloss:0.52963                             
[3]	test-logloss:0.52874                             
[4]	test-logloss:0.52711                             
[5]	test-logloss:0.52828                             
[6]	test-logloss:0.52926                             
[7]	test-logloss:0.52770                             
[8]	test-logloss:0.52813                             
[9]	test-logloss:0.52890                             
[0]	test-logloss:0.65375                                                       
[1]	test-logloss:0.65348                                                       
[2]	test-logloss:0.65321                                                       
[3]	test-logloss:0.65295                                                       
[4]	test-logloss:0.65268                                                       
[5]	test-logloss:0.65241                                    

In [7]:
# save the model with the best params
artifacts_path = "./mlflow"
tags = {
        "author": "andrei lupascu",
        "mode": CONFIG["mode"]
}
# update best_result
best_result["objective"] = search_space["objective"]
best_result["seed"] = search_space["seed"]
# format best results (some params need to be cast as int)
int_params = ["max_depth"]
for int_param in int_params:
        best_result[int_param] = int(best_result[int_param])

supports.objective(
     search_space=best_result,
     xtrain=xtrain,
     xtest=xtest,
     ytrain=ytrain,
     ytest=ytest,
     num_boost_round=CONFIG["booster_rounds"],
     tags=tags,
     save_artifacts=(True, artifacts_path, dv)
     )
    

[0]	test-logloss:0.55135
[1]	test-logloss:0.53217
[2]	test-logloss:0.52963
[3]	test-logloss:0.52874
[4]	test-logloss:0.52711
[5]	test-logloss:0.52828
[6]	test-logloss:0.52926
[7]	test-logloss:0.52770
[8]	test-logloss:0.52813
[9]	test-logloss:0.52890


/home/adi/miniconda3/envs/mlops/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [06:53:51] WARNING: /workspace/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)


{'loss': 0.5288979563958918, 'status': 'ok'}

In [2]:
from evidently import ColumnMapping
from evidently.report import Report
from evidently.metrics import ColumnDriftMetric, DatasetDriftMetric

In [11]:
prediction_setup = load_transform_predict.Predictor()
live_data = prediction_setup.live_data.copy()

prod_batch = pd.read_parquet("data/batch_data/testing_batches/prod_data_batch_1.parquet")
new_batch = pd.read_parquet("data/batch_data/testing_batches/new_data_batch_1.parquet")
prod_batch.drop(columns="subject_id", inplace=True)
new_batch.drop(columns="subject_id", inplace=True)

prod_batch = prediction_setup.predict(prod_batch)
new_batch = prediction_setup.predict(new_batch)

In [43]:
cat_cols = [col for col in live_data.columns if live_data[col].dtype == "object"]
num_cols = [col for col in live_data.columns if live_data[col].dtype != "object"]
all_cols = cat_cols + num_cols

In [31]:
column_mapping = ColumnMapping(
    target=None,
    prediction="prediction",
    numerical_features=num_cols,
    categorical_features=cat_cols
    )

In [44]:
report = Report(metrics=[
    ColumnDriftMetric(column_name="prediction"),
    DatasetDriftMetric(columns=all_cols, drift_share=0.1)
    ])

In [85]:
test_data = [prod_batch, new_batch]
report.run(
    reference_data=live_data, 
    current_data=test_data[1], 
    column_mapping=column_mapping
    )
# report.show(mode="inline")

In [89]:
results = report.as_dict()

pred_drift_results = results["metrics"][0]
pred_drift_status = pred_drift_results["result"]["drift_detected"]
pred_drift_col = pred_drift_results["result"]["column_name"]
pred_drift_test = pred_drift_results["result"]["stattest_name"]
pred_drift_threshold = pred_drift_results["result"]["stattest_threshold"]
pred_drift_score = pred_drift_results["result"]["drift_score"]

dataset_drift_results = results["metrics"][1]
dataset_drift_status = dataset_drift_results["result"]["dataset_drift"]
dataset_drift_columns = dataset_drift_results["result"]["number_of_drifted_columns"]
dataset_drift_treshold = dataset_drift_results["result"]["drift_share"]
dataset_drift_share = dataset_drift_results["result"]["share_of_drifted_columns"]

prediction_drift_warning = f"""---Prediction Drift Detected: 
Target name: {pred_drift_col}
Test used: {pred_drift_test}
Test threshold: {pred_drift_threshold}
Test score: {pred_drift_score}
"""
dataset_drift_warning = f"""---Dataset Drift Detected: 
Columns drifted: {dataset_drift_columns}
Drift threshold: {dataset_drift_treshold}
Drift Share: {dataset_drift_share}
"""

if (pred_drift_status + dataset_drift_status) > 0:
    print(prediction_drift_warning)
    print()
    print(dataset_drift_warning)    

---Prediction Drift Detected: 
Target name: prediction
Test used: Wasserstein distance (normed)
Test threshold: 0.1
Test score: 0.20168579624732844


---Dataset Drift Detected: 
Columns drifted: 5
Drift threshold: 0.1
Drift Share: 0.15151515151515152

